In [1]:
from pyspark.sql import SparkSession
import opmDataset
from pandas import *
from mmtfPyspark.io import mmtfReader
from ipywidgets import interact, IntSlider
import py3Dmol

#### Configure Spark

In [2]:
spark = SparkSession.builder.master("local[4]").appName("OPM_DEMO").getOrCreate()
sc = spark.sparkContext

In [3]:
dataset = opmDataset.get_dataset()

In [4]:
dataset.toPandas().head()

,Family,ProteinName,PDBID,Species,Localization,Num.Subunits,Num.TMSec.Structs.,HydrophobicThicknessorDepth(Å),TiltAngle(°),ΔGtransfer(kcal/mol)
0,1.1.01.01.,Cruxrhodopsin-3,4JR8,Haloarcula vallismortis,Archaebac.,3,21,31.8 ± 1.2,0 ± 0,-130.4
1,1.1.01.01.,"Bacteriorhodopsin, K state, with lipids",1IW6,Halobacterium salinarum,Archaebac.,3,21,30.0 ± 0.6,0 ± 0,-112.6
2,1.1.01.01.,"Bacteriorhodopsin, trimer, complex with annula...",2ZZL,Halobacterium salinarum,Archaebac.,3,21,29.8 ± 0.6,0 ± 0,-119.0
3,1.1.01.01.,Bacteriorhodopsin-I,4PXK,Haloarcula marismortui,Archaebac.,1,7,31.8 ± 1.4,11 ± 1,-72.7
4,1.1.01.01.,Bacteriorhodopsin,5AZD,Thermus thermophilus,Bact. Gram-neg inner,4,28,30.0 ± 0.5,3 ± 1,-121.3


In [21]:
# get pdb ids for alpha-helical transmembrane proteins and beta-barrel transmembrane proteins
opm_df = dataset.toPandas()
tm_alpha_pdbids = opm_df[[s.startswith('1.1.') for s in opm_df['Family']]]['PDBID'].tolist()
beta_barrel_pdbids = opm_df[[s.startswith('1.3.') for s in opm_df['Family']]]['PDBID'].tolist()

import urllib

def DepPDBFilter(pdbid):
    request = urllib.request.Request('https://mmtf.rcsb.org/v1.0/full/' + pdbid)
    request.get_method = lambda: 'HEAD'
    try:
        urllib.request.urlopen(request)
        return True
    except urllib.request.HTTPError:
        return False

cleanpdblist = [i for i in pdbids if DepPDBFilter(i)]
print(cleanpdblist)

In [12]:
#sppdbids = sc.parallelize(set(pdbids)).filter(lambda t: DepPDBFilter(t))
# structures = mmtfReader.download_full_mmtf_files(cleanpdblist, sc).cache()

TypeError: 'PipelinedRDD' object is not subscriptable

In [23]:
def view_proteins(pdb_ids):
    import pycurl
    def view3d(i=0):
        pdb_id = pdb_ids[i].lower()
        opm_url = 'http://opm.phar.umich.edu/pdb/'
        c = pycurl.Curl()
        c.setopt(c.URL, opm_url + pdb_id + '.pdb')
        with open(pdb_id + '.pdb', 'wb') as f:
            c.setopt(c.WRITEDATA, f)
            c.perform()
        
        print(f'PDB: {pdb_ids[i]}')

        structure = open(pdb_id + '.pdb', 'r').read()
        viewer = py3Dmol.view()
        viewer.addModel(structure, 'pdb')

        viewer.setStyle({'cartoon': {'color': 'spectrum'}})
        viewer.setStyle({'hetflag': True}, {'stick': {}})
        viewer.setStyle({'resn': 'DUM'}, {'sphere': {'radius': 0.3}})
        viewer.zoomTo()
        viewer.rotate(-90, 'x', 1)
        return viewer.show()

    s_widget = IntSlider(min=0, max=len(pdb_ids)-1, description='Structure', continuous_update=False)
    return interact(view3d, i=s_widget)

In [25]:
view_proteins(tm_alpha_pdbids)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Structure', max=1228), Output()…

<function __main__.view_proteins.<locals>.view3d(i=0)>

In [24]:
view_proteins(beta_barrel_pdbids)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Structure', max=241), Output())…

<function __main__.view_proteins.<locals>.view3d(i=0)>

In [ ]:
spark.stop()